## 1. import dataset from google data bucket

In [19]:
df = spark.read.csv("gs://mit805_data_bucket/data_raw/final_animedataset.csv", header = True, inferSchema=True)


In [24]:
df.show()

+--------+--------+--------+-------+------+--------------------+----+------------+-----+---------+------+----------+--------------------+
|username|anime_id|my_score|user_id|gender|               title|type|      source|score|scored_by|  rank|popularity|               genre|
+--------+--------+--------+-------+------+--------------------+----+------------+-----+---------+------+----------+--------------------+
|karthiga|      21|       9|2255153|Female|           One Piece|  TV|       Manga| 8.54| 423868.0|  91.0|      35.0|Action, Adventure...|
|karthiga|      59|       7|2255153|Female|             Chobits|  TV|       Manga| 7.53| 175388.0|1546.0|     188.0|Sci-Fi, Comedy, D...|
|karthiga|      74|       7|2255153|Female|        Gakuen Alice|  TV|       Manga| 7.77|  33244.0| 941.0|    1291.0|Comedy, School, S...|
|karthiga|     120|       7|2255153|Female|       Fruits Basket|  TV|       Manga| 7.77| 167968.0| 939.0|     222.0|Slice of Life, Co...|
|karthiga|     178|       7|225515

In [2]:
display(df)

DataFrame[username: string, anime_id: int, my_score: int, user_id: int, gender: string, title: string, type: string, source: string, score: string, scored_by: double, rank: double, popularity: double, genre: string]

In [5]:
# 2. Basic Statistics
df.describe().show()

+-------+-------------+------------------+------------------+------------------+----------+------------------+--------------------+------------+------------------+------------------+------------------+-----------------+------------------+
|summary|     username|          anime_id|          my_score|           user_id|    gender|             title|                type|      source|             score|         scored_by|              rank|       popularity|             genre|
+-------+-------------+------------------+------------------+------------------+----------+------------------+--------------------+------------+------------------+------------------+------------------+-----------------+------------------+
|  count|     35305439|          35305695|          35305695|          35305695|  35305695|          35305695|            35305695|    35305695|          35305695|          35305695|          34554095|         35305325|          35303428|
|   mean|     Infinity|11481.590444006271| 4

## 2. Checking for Missing Values
Determine the count of missing values in each column:

In [7]:
from pyspark.sql.functions import isnan, when, count, col

missing_data = df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns])
missing_data.show()


+--------+--------+--------+-------+------+-----+----+------+-----+---------+------+----------+-----+
|username|anime_id|my_score|user_id|gender|title|type|source|score|scored_by|  rank|popularity|genre|
+--------+--------+--------+-------+------+-----+----+------+-----+---------+------+----------+-----+
|     256|       0|       0|      0|     0|    0|   0|     0|    0|        0|751600|       370| 2267|
+--------+--------+--------+-------+------+-----+----+------+-----+---------+------+----------+-----+



In [7]:
#Determine number of unique Anime Titles
unique_titles =df.select("title").distinct().rdd.flatMap(lambda x: x).collect()
len(unique_titles)

8746

In [10]:
# Count the number of unique titles that do not have a rank value in the "rank" column
missing_rank_count = df.filter(df["rank"].isNull()).select("title").distinct().count()
missing_rank_count

1107

## 3. Handling Missing Values

In [6]:
# 3. Checking for Missing Values
for col in df.columns:
    print(col, ":", df.filter(df[col].isNull()).count())

username : 256


anime_id : 0


my_score : 0


user_id : 0


gender : 0


...

score : 0


scored_by : 0


rank : 751600


popularity : 370


genre : 2267


In [ ]:
#Drop rows where any of the specified columns have missing values
cleaned_data = data.dropna(subset=["genre", "popularity", "username"])

In [8]:
# 4. Distribution of User Scores
df.groupBy("my_score").count().orderBy("my_score").show()

+--------+--------+
|my_score|   count|
+--------+--------+
|       0|13743594|
|       1|  137481|
|       2|  165239|
|       3|  277507|
|       4|  586871|
|       5| 1318123|
|       6| 2514152|
|       7| 4822985|
|       8| 5329087|
|       9| 3721491|
|      10| 2689165|
+--------+--------+



## 4. Top Rated Animes

In [2]:
top_rated = df.select("title", "score", "scored_by").distinct().orderBy("score", ascending=False)
top_rated.show(10)

+--------------------+------------+---------+
|               title|       score|scored_by|
+--------------------+------------+---------+
|"Marriage Blue: "...|Visual novel|     6.42|
|"Gyakuten Saiban:...|        Game|     6.44|
|Fullmetal Alchemi...|        9.25| 733592.0|
|      Kimi no Na wa.|        9.19| 471398.0|
|       Steins;Gate 0|        9.15|  26739.0|
|            Gintama°|        9.15|  71751.0|
|         Steins;Gate|        9.14| 563857.0|
|Hunter x Hunter (...|        9.11| 403377.0|
|Ginga Eiyuu Densetsu|        9.11|  29036.0|
|       Gintama&#039;|        9.11|  92025.0|
+--------------------+------------+---------+
only showing top 10 rows



In [4]:
top_rated.write.save("gs://mit805_data_bucket/Processed/top_rated.csv",format="csv")

In [10]:
# 6. Most Active Users
df.groupBy("username").count().orderBy("count", ascending=False).show(10)


+-------------+-----+
|     username|count|
+-------------+-----+
|  spacecowboy| 8745|
|   TsukasaKei| 8745|
|      uemmega| 8594|
|       Exxorn| 8562|
| DeadlyKizuna| 8159|
|       xbhrjd| 8084|
|   JakCooper2| 7897|
|      De_Baer| 7414|
|   Dedzapadlo| 7239|
|DesireDestiny| 7012|
+-------------+-----+
only showing top 10 rows



In [11]:
# 7. Distribution by Type and Source
df.groupBy("type").count().show()


+--------------------+--------+
|                type|   count|
+--------------------+--------+
|                  TV|23696077|
|           Igi Ari!"|    7790|
| Doushite Konna O...|     370|
|             Special| 2576837|
|                 OVA| 3952868|
|               Music|  176054|
|               Movie| 4270626|
|                 ONA|  625073|
+--------------------+--------+



In [12]:
df.groupBy("source").count().show()


+-------------+--------+
|       source|   count|
+-------------+--------+
|           TV|    7790|
| Visual novel| 2462047|
|Digital manga|   17600|
|     Original| 6400588|
|        Novel| 1320645|
| Picture book|   12522|
|         Book|   68975|
|      Unknown| 1402138|
|        Other|  413565|
|        Radio|    2882|
|        Manga|14655746|
| 4-koma manga| 1008246|
|          OVA|     370|
|        Music|   96945|
|         Game| 1270683|
|    Web manga|  509959|
|    Card game|   88883|
|  Light novel| 5566111|
+-------------+--------+



ERROR:root:Exception while sending command.                                     
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/lib/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
/usr/lib/spark/python/pyspark/context.py:561: RuntimeWarning: Unable to cleanly shutdown Spark JVM process. It is possible that the process has crashed, been killed or may also be in a zombie state.

In [ ]:
## Using 

In [10]:
rdd =sc.textFile("gs://mit805_data_bucket/data_raw/final_animedataset.csv")

# Extract the title (assuming title is the first column) and emit with count 1
mapped_rdd = rdd.map(lambda line: (line.split(",")[0], 1))

# Aggregate counts for each title
reduced_rdd = mapped_rdd.reduceByKey(lambda a, b: a + b)

# Get the most popular anime by sorting the counts
most_popular_anime = reduced_rdd.sortBy(lambda x: x[1], ascending=False).first()
most_popular_anime 

('spacecowboy', 8745)

# Operation using Map Reduce

## Missing Values

In [11]:
rdd =sc.textFile("gs://mit805_data_bucket/data_raw/final_animedataset.csv")

# Extract headers (column names) first
header = rdd.first()
columns = header.split(",")

# Filter out the header for further processing
rdd_no_header = rdd.filter(lambda line: line != header)

def map_missing_values(line):
    values = line.split(",")
    # Check if the length of values matches the number of columns
    if len(values) != len(columns):
        return []
    return [(columns[i], 1 if value.strip() == "" else 0) for i, value in enumerate(values)]

mapped_rdd = rdd_no_header.flatMap(map_missing_values)

# Aggregate counts for missing values for each column
missing_values_count = mapped_rdd.reduceByKey(lambda a, b: a + b).collect()
missing_values_count

[('gender', 0),
 ('my_score', 0),
 ('scored_by', 0),
 ('user_id', 0),
 ('source', 0),
 ('popularity', 0),
 ('anime_id', 0),
 ('rank', 431872),
 ('score', 0),
 ('genre', 2267),
 ('title', 0),
 ('username', 9),
 ('type', 0)]

## Number of uniqe anime titles

In [12]:
# Extract headers (column names) first
header = rdd.first()
columns = header.split(",")

# Assuming title is the first column, extract titles
title_index = columns.index("title")
titles_rdd = rdd_no_header.map(lambda line: line.split(",")[title_index])

# Get unique titles and count them
unique_titles_count = titles_rdd.distinct().count()
unique_titles_count

8725

## Distribution by source and type

In [15]:
# Assuming rdd_no_header is the RDD without the header row

# Extract and count the types
type_index = columns.index("type")
type_mapped_rdd = rdd_no_header.map(lambda line: (line.split(",")[type_index], 1))
type_counts_rdd = type_mapped_rdd.reduceByKey(lambda a, b: a + b)

# Order by count (descending)
type_counts_sorted = type_counts_rdd.sortBy(lambda x: x[1], ascending=False).collect()

# Extract and count the sources
source_index = columns.index("source")
source_mapped_rdd = rdd_no_header.map(lambda line: (line.split(",")[source_index], 1))
source_counts_rdd = source_mapped_rdd.reduceByKey(lambda a, b: a + b)

# Order by count (descending)
source_counts_sorted = source_counts_rdd.sortBy(lambda x: x[1], ascending=False).collect()


In [16]:
source_counts_sorted

[('Manga', 14563148),
 ('Original', 6334432),
 ('Light novel', 5470223),
 ('Visual novel', 2374314),
 ('Unknown', 1380961),
 ('Novel', 1275539),
 ('Game', 1265982),
 ('4-koma manga', 1002121),
 ('Web manga', 498274),
 ('Other', 409295),
 ('TV', 188722),
 ('Music', 96710),
 ('Special', 96339),
 ('Card game', 88505),
 ('OVA', 79605),
 ('Book', 68636),
 ('Movie', 43459),
 ('Digital manga', 14792),
 ('Picture book', 12214),
 (' Chougekisen"', 11474),
 (' Koi Iroha."', 10547),
 (' Imouto', 4052),
 ('Radio', 2882),
 (' Mondaiji."', 2808),
 (' Koi Iroha.: Inari', 2715),
 (' Tokimeki', 2442),
 (' Jump!"', 2009),
 ('ONA', 1591),
 (' Mata Itsuka Utattari"', 794),
 (' Socchi"', 308),
 (' de arimasu!!"', 284),
 (' Commando"', 215),
 (' Alive"', 173),
 (' Dance"', 130)]

In [18]:
type_counts_sorted

[('TV', 23501482),
 ('Movie', 4215194),
 ('OVA', 3866693),
 ('Special', 2477262),
 ('ONA', 623482),
 ('Music', 172044),
 (' Tomoyo-hen"', 27329),
 (' Where We Are Least Alone."', 25931),
 (' Kaku Tatakaeri"', 25373),
 (' Kyou-hen"', 23053),
 (' Gakuen Love Comedy wo Zenryoku de Jama Shiteiru"', 21082),
 (' Yakusoku no Basho"', 20775),
 (' H ga Dekinai."', 20692),
 (' Kaku Tatakaeri 2nd Season"', 17406),
 (' Imouto ga Iru!"', 14002),
 (' Konkon', 13262),
 (' Ressen', 11474),
 (' Soko ni Iru Boku"', 11053),
 (' Imouto no Yousu ga Chotto Okashiinda ga."', 10862),
 (' Kougyou"', 9780),
 (' Twintail ni Narimasu."', 8710),
 (' Minotake ni Attemasu"', 8112),
 (' Azazel-san. (TV)"', 7843),
 (' Igi Ari!"', 7790),
 (' Koi ka Ken... ""Naku na"""', 6959),
 (' Nazomeite Miru"', 6297),
 (' Kiseki no Sakura"', 5972),
 (' Tsutsumimasu!"', 5964),
 (' Gakuen Love Comedy wo Zenryoku de Jama Shiteiru OVA"', 5523),
 (' White"', 5388),
 (' Girls!"', 5313),
 (' H ga Dekinai. OVA"', 5084),
 (' UFO no Natsu"',